# Data Aggregation and Group Operations

In [1]:
import numpy as np
import pandas as pd

## GroupBy

In [2]:
df = pd.DataFrame({'producto':list('aabba'),
                    'vendedor':['Juan','Celia','Juan','Celia', 'Juan' ],
                      'balance': np.random.randn(5)*10,
                       'income': np.random.randn(5)+2,
                  })

In [3]:
df

,balance,income,producto,vendedor
0,-17.575465,1.020607,a,Juan
1,-2.123048,3.503198,a,Celia
2,-10.223210,1.875094,b,Juan
3,-0.808254,2.179792,b,Celia
4,-4.403535,1.144516,a,Juan


_Ejercicio de repaso_: sacar media de balance e income

In [5]:
df['balance'].mean()

-7.026702320594273

In [6]:
df['income'].mean()

1.944641263981696

Para sacar la media por producto

In [9]:
means = df.groupby('producto').mean()
means

,balance,income
producto,,
a,-8.034016,1.889440
b,-5.515732,2.027443


In [11]:
type(df.groupby('producto'))

pandas.core.groupby.DataFrameGroupBy

In [12]:
df.groupby('producto')

_NOTA: La salida de groupby es un Dataframe _especial_


In [14]:
mean_producto = df.groupby('producto')['balance'].mean()
mean_producto

producto
a   -8.034016
b   -5.515732
Name: balance, dtype: float64

In [15]:
type(mean_producto)

pandas.core.series.Series

Media del balance para el producto a

In [17]:
df.groupby('producto')['balance'].mean()['a']

-8.03401575678113

Se puede agrupar por varias columnas

In [18]:
df.groupby(['producto', 'vendedor']).mean()

balance    income
producto vendedor                     
a        Celia     -2.123048  3.503198
         Juan     -10.989500  1.082561
b        Celia     -0.808254  2.179792
         Juan     -10.223210  1.875094

Se pueden hacer varias operaciones a la vez con el grupo

In [19]:
df.groupby(['producto', 'vendedor']).agg(['mean', 'count'])

balance          income      
                        mean count      mean count
producto vendedor                                 
a        Celia     -2.123048     1  3.503198     1
         Juan     -10.989500     2  1.082561     2
b        Celia     -0.808254     1  2.179792     1
         Juan     -10.223210     1  1.875094     1

In [22]:
def strseries(serie):
    return serie.str.len().sum()

In [28]:
df.groupby('producto')['producto','vendedor'].head()

,producto,vendedor
0,a,Juan
1,a,Celia
2,b,Juan
3,b,Celia
4,a,Juan


In [26]:
df.groupby('producto')['vendedor'].agg(strseries)

producto
a    13
b     9
Name: vendedor, dtype: int64

_Nota:_ esto hace ds grupos, uno para el producto a , de tres filas y otro para el b, de 2. De esos grupos sacamos el vendedor, que será, respectivamente, una seria de 3 y 2 vendedores. Y de ahí las agrego usando la función strseries que suma el número de letras de la serie.

## Iterando sobre grupos

Me hago una función para imprimir grupos

In [36]:
def print_groups(groups):
    for key,group in groups:
        print(f"Clave '{key}' ")
        print(group)

In [37]:
print_groups(df.groupby('producto'))

Clave 'a' 
     balance    income producto vendedor
0 -17.575465  1.020607        a     Juan
1  -2.123048  3.503198        a    Celia
4  -4.403535  1.144516        a     Juan
Clave 'b' 
     balance    income producto vendedor
2 -10.223210  1.875094        b     Juan
3  -0.808254  2.179792        b    Celia


Otra forma de ver grupos o de convertirlo

In [38]:
list(df.groupby('producto'))

[('a',      balance    income producto vendedor
  0 -17.575465  1.020607        a     Juan
  1  -2.123048  3.503198        a    Celia
  4  -4.403535  1.144516        a     Juan),
 ('b',      balance    income producto vendedor
  2 -10.223210  1.875094        b     Juan
  3  -0.808254  2.179792        b    Celia)]

In [40]:
dict(list(df.groupby('producto')))

{'a':      balance    income producto vendedor
 0 -17.575465  1.020607        a     Juan
 1  -2.123048  3.503198        a    Celia
 4  -4.403535  1.144516        a     Juan,
 'b':      balance    income producto vendedor
 2 -10.223210  1.875094        b     Juan
 3  -0.808254  2.179792        b    Celia}

In [ ]:
type(cuentas['a'])

In [41]:
cuentas['a']['balance']

0   -17.575465
1    -2.123048
4    -4.403535
Name: balance, dtype: float64

##  Data aggregation

In [44]:
import requests
url = 'https://raw.githubusercontent.com/wesm/pydata-book/1st-edition/ch08/tips.csv'
response = requests.get(url)

out_file = open('tips.csv','wb')
out_file.write(response.content)
out_file.close()

In [45]:
tips = pd.read_csv('tips.csv')
tips.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


In [46]:
tips.count()

total_bill    244
tip           244
sex           244
smoker        244
day           244
time          244
size          244
dtype: int64

_Ejercicio_: Obtener el porcentaje de propina y analizarlo según sexo y fumador

In [47]:
tips['percentage'] = 100 * tips['tip'] / tips['total_bill'] 

In [48]:
tips.head()

,total_bill,tip,sex,smoker,day,time,size,percentage
0,16.99,1.01,Female,No,Sun,Dinner,2,5.944673
1,10.34,1.66,Male,No,Sun,Dinner,3,16.054159
2,21.01,3.50,Male,No,Sun,Dinner,3,16.658734
3,23.68,3.31,Male,No,Sun,Dinner,2,13.978041
4,24.59,3.61,Female,No,Sun,Dinner,4,14.680765


In [53]:
tips.groupby(['sex'])['percentage'].describe()

,count,mean,std,min,25%,50%,75%,max
sex,,,,,,,,
Female,87.0,16.649074,5.363173,5.643341,14.041645,15.558149,19.426621,41.666667
Male,157.0,15.765055,6.477787,3.563814,12.138869,15.349194,18.623962,71.034483


In [54]:
tips.groupby(['smoker'])['percentage'].describe()

,count,mean,std,min,25%,50%,75%,max
smoker,,,,,,,,
No,151.0,15.932846,3.990977,5.679667,13.690561,15.562472,18.501403,29.198966
Yes,93.0,16.319604,8.511920,3.563814,10.677083,15.384615,19.505852,71.034483


In [52]:
tips.groupby(['sex', 'smoker'])['percentage'].describe()

count       mean       std       min        25%        50%  \
sex    smoker                                                               
Female No       54.0  15.692097  3.642118  5.679667  13.970835  14.969118   
       Yes      33.0  18.215035  7.159451  5.643341  15.243902  17.391304   
Male   No       97.0  16.066872  4.184875  7.180385  13.181019  15.760441   
       Yes      60.0  15.277118  9.058794  3.563814  10.184496  14.101483   

                     75%        max  
sex    smoker                        
Female No      18.162966  25.267250  
       Yes     19.821606  41.666667  
Male   No      18.621974  29.198966  
       Yes     19.169707  71.034483

In [55]:
tips.groupby(['size'])['percentage'].describe()

,count,mean,std,min,25%,50%,75%,max
size,,,,,,,,
1,4.0,21.729202,8.034186,13.793103,17.077869,20.275206,24.926539,32.573290
2,156.0,16.571919,6.684824,3.563814,13.522313,15.610418,19.503614,71.034483
3,38.0,15.215685,4.545887,5.643341,12.475755,15.932311,18.613473,23.074192
4,37.0,14.594901,4.239533,7.745933,11.774956,14.669927,16.979656,28.053517
5,5.0,14.149549,6.773266,6.565988,10.657194,12.138869,17.219430,24.166264
6,4.0,15.622920,4.215338,10.379905,13.165446,16.289124,18.746598,19.533528


Para analizar, hay que tener en cuenta:
- si cada grupo es suficientemente grande. 
- ver si la diferencia en media entre grupos es signifitcatida, teniendo en cuenta también la desviación media, que en el fondo es el error

Creo funciones de agregración:

In [ ]:
Diferencia entre máximo y mínimo:

In [66]:
def peak_to_peak(serie):
    return serie.max() - serie.min()

    

In [ ]:
Rango normal del 95% (4 veces la desviación estandar)

In [65]:
def rango_normal(serie):
    return 4*serie.std()

Se pueden usar nuestras propieads funciones. Tambíén se pueden renombrar columnas

In [69]:
tips.groupby(['sex', 'smoker'])['percentage'].agg([('media','mean'),('desv. std.','std'), 'count', ('rango', peak_to_peak), ('rango 95%', rango_normal) ])

media  desv. std.  count      rango  rango 95%
sex    smoker                                                    
Female No      15.692097    3.642118     54  19.587583  14.568474
       Yes     18.215035    7.159451     33  36.023326  28.637805
Male   No      16.066872    4.184875     97  22.018581  16.739501
       Yes     15.277118    9.058794     60  67.470669  36.235176

_Ejercicio_: hemos visto que la gente es muy extremista. Vamos a analizar esos extremos. Extraer los casos que tengan una propina superior al 40%

In [71]:
tips[tips['percentage'] > 40]

,total_bill,tip,sex,smoker,day,time,size,percentage
172,7.25,5.15,Male,Yes,Sun,Dinner,2,71.034483
178,9.60,4.00,Female,Yes,Sun,Dinner,2,41.666667


Un domingo, una cena, en pareja... ejem

## Unstack

Sirve para cambiar la forma de _ordenar_ la información agrupada. Pasarla de las columnas a las filas. Es decir, para crear subcabeceras a partir de subcolumnas 


In [74]:
stacked = df.groupby(['producto', 'vendedor']).mean()

In [75]:
stacked

balance    income
producto vendedor                     
a        Celia     -2.123048  3.503198
         Juan     -10.989500  1.082561
b        Celia     -0.808254  2.179792
         Juan     -10.223210  1.875094

In [79]:
stacked.unstack('vendedor')

balance              income          
vendedor     Celia      Juan     Celia      Juan
producto                                        
a        -2.123048 -10.98950  3.503198  1.082561
b        -0.808254 -10.22321  2.179792  1.875094

_Nota_: Es solo otra forma de presentar los datos: llevamos vendedor de las filas a las columnas

In [80]:
stacked.unstack('vendedor').columns

MultiIndex(levels=[['balance', 'income'], ['Celia', 'Juan']],
           labels=[[0, 0, 1, 1], [0, 1, 0, 1]],
           names=[None, 'vendedor'])

_Nota:_ esto nos da información sobre las columnas y sus etiquetas 

Podemos hacer unstack de varias columnas, lo que pasa es que _balance_ ya era una columna

In [83]:
stacked.unstack('vendedor', 'balance')

balance              income          
vendedor     Celia      Juan     Celia      Juan
producto                                        
a        -2.123048 -10.98950  3.503198  1.082561
b        -0.808254 -10.22321  2.179792  1.875094

Ahora lo vamos a hacer para el vendedor Celia, y tenemos que hacer la media por cada producto vendido

In [90]:
df[df['vendedor'] == "Celia"].groupby('producto').mean()

,balance,income
producto,,
a,-2.123048,3.503198
b,-0.808254,2.179792


In [91]:
df[df['vendedor'] == "Celia"].groupby('producto').mean().unstack('producto')

         producto
balance  a          -2.123048
         b          -0.808254
income   a           3.503198
         b           2.179792
dtype: float64

Se convierte en una especia de serie

###  Pivot

Para crear subcabeceras a partir de una cabecera

In [94]:
df

,balance,income,producto,vendedor
0,-17.575465,1.020607,a,Juan
1,-2.123048,3.503198,a,Celia
2,-10.223210,1.875094,b,Juan
3,-0.808254,2.179792,b,Celia
4,-4.403535,1.144516,a,Juan


In [93]:
df.pivot(columns='producto')

balance               income           vendedor       
producto          a          b         a         b        a      b
0        -17.575465        NaN  1.020607       NaN     Juan   None
1         -2.123048        NaN  3.503198       NaN    Celia   None
2               NaN -10.223210       NaN  1.875094     None   Juan
3               NaN  -0.808254       NaN  2.179792     None  Celia
4         -4.403535        NaN  1.144516       NaN     Juan   None

### Example: filling missin values with group-specific values

In [95]:
states = ['Ohio', 'New York', 'Vermont', 'Florida', 'Oregon', 'Nevada', 'California', 'Idaho']

In [97]:
df_us = pd.DataFrame(
{
    'states':states,
        'market': ['East'] * 4 + ['West'] * 4,
       'data': [100,82,83,np.nan,20,30,np.nan,np.nan],
})

In [98]:
df_us

,data,market,states
0,100.0,East,Ohio
1,82.0,East,New York
2,83.0,East,Vermont
3,NaN,East,Florida
4,20.0,West,Oregon
5,30.0,West,Nevada
6,NaN,West,California
7,NaN,West,Idaho


Rellenamos los datos vacíos de data con la media de data

In [102]:
print_groups(df_us.groupby('market')['data'])

Clave 'East' 
0    100.0
1     82.0
2     83.0
3      NaN
Name: data, dtype: float64
Clave 'West' 
4    20.0
5    30.0
6     NaN
7     NaN
Name: data, dtype: float64


In [100]:
df_us.groupby('market')['data'].apply(lambda serie_de_data: serie_de_data.fillna(serie_de_data.mean()))

0    100.000000
1     82.000000
2     83.000000
3     88.333333
4     20.000000
5     30.000000
6     25.000000
7     25.000000
Name: data, dtype: float64

Me liaba porque en el fonde esto no es una serie, sino la dataframe esta especial del group by. Este función lo apica a todas los valores de todos los grupo s y te lo da en una serie plana: Ver: https://pandas.pydata.org/pandas-docs/stable/generated/pandas.core.groupby.GroupBy.apply.html

In [105]:
df_us2 = df_us.copy()

In [106]:
df_us2['data'] = df_us2.groupby('market')['data'].apply(lambda serie_de_data: serie_de_data.fillna(serie_de_data.mean()))

In [107]:
df_us2

,data,market,states
0,100.000000,East,Ohio
1,82.000000,East,New York
2,83.000000,East,Vermont
3,88.333333,East,Florida
4,20.000000,West,Oregon
5,30.000000,West,Nevada
6,25.000000,West,California
7,25.000000,West,Idaho


Ahora lo vamos a intentar llenar con un diccionario

In [110]:
fill_values = { 'East': 10, 'West': 200}

In [109]:
fill_func = lambda group : group.fillna(fill_values[group.name])

In [111]:
df_us.groupby('market').apply(fill_func)

,data,market,states
0,100.0,East,Ohio
1,82.0,East,New York
2,83.0,East,Vermont
3,10.0,East,Florida
4,20.0,West,Oregon
5,30.0,West,Nevada
6,200.0,West,California
7,200.0,West,Idaho
